In [1]:
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [3]:
df = pl.read_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora_komplet').str.contains('Ústavod') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
)

In [69]:
len(df)

722809

In [75]:
memory_bytes = df.estimated_size()
print(f"Memory usage: {memory_bytes / 1024**2:.2f} MB")

Memory usage: 983.28 MB


In [4]:
df.select(pl.col("komora")).unique().to_series().to_list()

['cnr', 'ns', 'pns', 'fs', 'ps']

In [5]:
df.filter(pl.col("komora").str.contains("psse"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool


In [6]:
df.select(pl.col("komora_komplet")).unique().to_series().to_list()

['Prozatimní NS RČS 1945-1946',
 'PČR, PS 2006-2010',
 'Národní shromáždění československé / Poslanecká sněmovna1920 – 1925, Poslanecká sněmovna',
 'Parlament České republiky, Poslanecká sněmovna 1996 – 1998',
 'Parlament České republiky, Poslanecká sněmovna 2006 – 2010',
 'Národní shromáždění československé / Poslanecká sněmovna1925 – 1929, Poslanecká sněmovna',
 'Parlament České republiky, Poslanecká sněmovna 2010 – 2013',
 'NS RČS 1960-1964',
 'Parlament České republiky, Poslanecká sněmovna 1998 – 2002',
 'Parlament České republiky, Poslanecká sněmovna 1993 – 1996',
 'ČNR 1976-1981',
 'NS RČS 1964-1968',
 'Česká národní rada 1986 – 1990',
 'NS RČS 1954-1960',
 'NS RČS 1929-1935, Poslanecká sněmovna',
 'FS ČSSR 1971-1976',
 'Parlament České republiky, Poslanecká sněmovna 2002 – 2006',
 'Parlament České republiky, Poslanecká sněmovna 2013 – 2017',
 'PČR, PS 2021-...',
 'Federální shromáždění ČSSR, 1981 – 1986 Sněmovna lidu',
 'Česká národní rada 1990 – 1992',
 'PČR, PS 2017-...',
 'Če

In [7]:
df.sample()

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool
"""Předseda PSP Jan Hamáček""","""2013ps_stenprot_057schuz_s057162.htm""","""/sqw/detail.sqw?id=5462""","""Děkuji paní poslankyni. A prosím pana poslance Zavadila, který je posledním řádně přihlášeným do obecné rozpravy.""",5,"""ps""",2013,57,162,2017-05-23,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""57. schůze""",true,true,true


In [8]:
import re

In [9]:
def najdi_lexy(retezec):
    return [x.replace('lex','Lex').replace('Ukrajinu','Ukrajina').replace('Ukrajině','Ukrajina').replace('Ukrajinou','Ukrajina').replace('Ukrajiny','Ukrajina').replace('plynu','plyn') for x in re.findall(r'[Ll]ex \w{1,20}-{0,1}\w{1,20}',retezec)]

In [10]:
najdi_lexy('Přijali jsme lex Babiš, pak Lex Švancnberk, lex Pat-Mat, to bylo z lexů vše.')

['Lex Babiš', 'Lex Švancnberk', 'Lex Pat-Mat']

In [11]:
lexy = df.filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)
lexy

C:\Users\micha\AppData\Local\Temp\ipykernel_1168\901567343.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  lexy = df.filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)


lex,len
str,u32
"""Lex Ukrajina""",600
"""Lex Babiš""",169
"""Lex specialis""",86
"""Lex OZE""",64
"""Lex ČEZ""",32
"""Lex Dukovany""",31
"""Lex Cyril""",27
"""Lex esto""",22
"""Lex covid""",22


In [26]:
lexy21 = df.filter(pl.col("obdobi") == 2021).filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)
lexy21

C:\Users\micha\AppData\Local\Temp\ipykernel_1168\3968080234.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  lexy21 = df.filter(pl.col("obdobi") == 2021).filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)


lex,len
str,u32
"""Lex Ukrajina""",600
"""Lex OZE""",64
"""Lex Babiš""",64
"""Lex ČEZ""",32
"""Lex specialis""",20
"""Lex plyn""",19
"""Lex Pawlas""",6
"""Lex dvojčata""",4
"""Lex covid""",4


In [30]:
pomluvcich = df.filter(pl.col("obdobi") == 2021).filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").agg(pl.col("mluvci").unique().len()).sort(by="mluvci",descending=True)
pomluvcich

C:\Users\micha\AppData\Local\Temp\ipykernel_1168\2506151919.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  pomluvcich = df.filter(pl.col("obdobi") == 2021).filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").agg(pl.col("mluvci").unique().len()).sort(by="mluvci",descending=True)


lex,mluvci
str,u32
"""Lex Ukrajina""",63
"""Lex OZE""",21
"""Lex plyn""",13
"""Lex specialis""",13
"""Lex Babiš""",7
"""Lex ČEZ""",7
"""Lex Pawlas""",3
"""Lex Vrbětice""",3
"""Lex Národní""",3


In [31]:
pomluvcich.filter(pl.col("mluvci") == 1)

lex,mluvci
str,u32
"""Lex Pošta""",1
"""Lex Michálek""",1
"""Lex obnovitelné""",1
"""Lex Brno""",1
"""Lex Blažek""",1
"""Lex atom""",1
"""Lex developer""",1
"""Lex esto""",1
"""Lex Ukrajin""",1


In [32]:
pomluvcich.filter(pl.col("mluvci") >= 2)

lex,mluvci
str,u32
"""Lex Ukrajina""",63
"""Lex OZE""",21
"""Lex plyn""",13
"""Lex specialis""",13
"""Lex Babiš""",7
"""Lex ČEZ""",7
"""Lex Pawlas""",3
"""Lex Vrbětice""",3
"""Lex Národní""",3


In [33]:
lexy21 = df.filter(pl.col("obdobi") == 2021).filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)
lexy.rename({"len":"n"})

C:\Users\micha\AppData\Local\Temp\ipykernel_1168\2386524876.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  lexy21 = df.filter(pl.col("obdobi") == 2021).filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)


lex,n
str,u32
"""Lex Ukrajina""",600
"""Lex Babiš""",169
"""Lex specialis""",86
"""Lex OZE""",64
"""Lex ČEZ""",32
"""Lex Dukovany""",31
"""Lex Cyril""",27
"""Lex esto""",22
"""Lex covid""",22


In [34]:
lexy.filter(pl.col("len") > 1)

lex,len
str,u32
"""Lex Ukrajina""",600
"""Lex Babiš""",169
"""Lex specialis""",86
"""Lex OZE""",64
"""Lex ČEZ""",32
"""Lex Dukovany""",31
"""Lex Cyril""",27
"""Lex esto""",22
"""Lex covid""",22


In [35]:
lexy.select(pl.col("len").median())

len
f64
1.0


In [36]:
lexy_top = lexy.drop_nulls().filter(pl.col('len') > 10).select(pl.col('lex')).to_series().to_list()
lexy_top

['Lex Ukrajina',
 'Lex Babiš',
 'Lex specialis',
 'Lex OZE',
 'Lex ČEZ',
 'Lex Dukovany',
 'Lex Cyril',
 'Lex esto',
 'Lex covid',
 'Lex plyn',
 'Lex Perek',
 'Lex Dérer',
 'Lex Baťa',
 'Lex posterior',
 'Lex Ofner',
 'Lex Uhlíř',
 'Lex generalis',
 'Lex Šumava']

In [37]:
poobdobi = df.with_columns(
    pl.col("text").str.count_matches("(?i) lex ").alias("lex")
).group_by("obdobi").agg(pl.col("lex").sum()).sort(by="obdobi")

poobdobi

obdobi,lex
i32,u32
1918,12
1920,56
1925,78
1929,38
1935,17
1945,2
1948,0
1954,0
1960,0


In [38]:
poobdobi.filter(pl.col('obdobi') < 2020).select(pl.col('lex').sum())

lex
u32
541


In [39]:
def hledej(text, n=3, delka=200):
    nalezy = df.sort(by="datum").filter(pl.col("text").str.contains(text)).head(n).iter_rows()
    for cislo, radek in enumerate(nalezy):
        print(f'Kdo: {radek[0]}')
        print(f'Kde: {radek[1]}')
        print(f'Kdy: {radek[9]}')
        
        # Split the text around the search term
        parts = re.split(text, radek[3])
        
        # Get the part before the search term (safe if empty)
        before = parts[0] if len(parts) > 0 else ""
        # Get the part after the search term (safe if empty)
        after = parts[1] if len(parts) > 1 else ""
        
        # Take up to 200 characters from before and after, but don't break if shorter
        before_snippet = before[-delka:] if len(before) > delka else before
        after_snippet = after[:delka] if len(after) > delka else after
        nalez = re.search(text, radek[3]).group()
        
        uryvek = before_snippet + nalez + after_snippet
        print(f'Co: {uryvek}')
        
        if cislo != 2:  # Changed from 3 to 2 since enumerate starts at 0
            print("***")

In [40]:
hledej("lex lečo")

Kdo: Poslanec Hubert Lang
Kde: 2021ps_stenprot_055schuz_s055125.htm
Kdy: 2023-03-01
Co: parlamentní kulturu. Já jsem dneska tedy ve dvě hodiny ráno skončil, měl jsem tady nějaké faktické poznámky, protože nejsem právník, tak se nemůžu vyjadřovat k právnímu stavu věci. Probrali jsme tady lex lečo, že jo, například, a další věci. Před nějakým časem jsem vysílal signály k panu premiérovi na Úřad vlády, který je nepřijal. Já prostě také nesouhlasím s tím, aby mi někdo omezil mojí jedinou šanci, 
***


In [41]:
hledej("lex dvojčata")

Kdo: Poslanec Šimon Heller
Kde: 2021ps_stenprot_078schuz_s078263.htm
Kdy: 2023-10-27
Co: Dámy a pánové, budu velmi krátký. Všechny vás chci naléhavě požádat o podporu lex dvojčata. Je to pozměňovací návrh, který napravuje historickou křivdu, kdy dneska dvojčata mají nárok pouze na 1,5násobek příspěvku rodičovské. Je to obrovská historická křivda. A tato Sněmovna má podruhé mož
***
Kdo: Poslanec Šimon Heller
Kde: 2021ps_stenprot_136schuz_s136036.htm
Kdy: 2025-04-16
Co: Dámy a pánové, obracím se na vás (Hlasitě.) - doufám, že jsem vás nevylekal - s návrhem, teď vás vylekám možná tím pozměňovacím návrhem, který se týká dvojčat, lex dvojčata. Jsem přesvědčen, že bychom měli narovnat práva dvojčat, a proto jsem načetl pozměňovací návrh, který napravuje historickou nespravedlnost, která tady je, protože stále stát považuje dvojčata za 1,5n
***
Kdo: Poslanec Šimon Heller
Kde: 2021ps_stenprot_136schuz_s136040.htm
Kdy: 2025-04-16
Co: Děkuji za slovo. Hlásím se k pozměňovacímu návrhu číslo 6473 

In [42]:
hledej("lex železářství")

Kdo: Poslanec Jakub Michálek
Kde: 2021ps_stenprot_041schuz_s041122.htm
Kdy: 2022-10-14
Co: ě Kocmanové zajišťuje, že bude vhodně implementován mechanismus filtrování, o němž samotném jsme vyjádřili určité pochybnosti. Další návrh, který jsme předložili s kolegou Hellerem, to je to takzvané lex železářství, nebo kdo chce kadeřnictví, nebo pan ministr má prodavačku, tak to je návrh, který skutečně navazuje na judikaturu Soudního dvora Evropské unie. Musím zde reagovat na obavy, které tady zazněly, že te
***


In [43]:
hledej("lex Brno")

Kdo: Poslanec Ivan Adamec
Kde: 2021ps_stenprot_053schuz_s053133.htm
Kdy: 2023-03-07
Co: o místní rozvoj, ty pozměňovací návrhy byly staženy anebo se neodsouhlasily. Na druhé straně, už jsem tam říkal, že mě trápí ještě jedna věc, a to je vyřešit, řeknu to velmi jednoduše, lex Dukovany a lex Brno, to znamená tu trubku z Dukovan do Brna, protože to je záležitost, ta myšlenka už je letitá, ale díky situaci, která tady nastala po ruské invazi a po energetické krizi, je ta myšlenka velmi aktuální
***


## Byl Lex Babiš první?

In [45]:
hledej("lex Babiš")

Kdo: Poslanec Miroslav Kalousek
Kde: 2013ps_stenprot_005schuz_s005004.htm
Kdy: 2014-01-21
Co: není ani slovo. Ale svou aktivní spoluprací s prezidentem republiky způsobíte, že od zítřka nebudou na ministry vyžadována lustrační osvědčení. Prezident republiky tohle zcela jasně řekl. To znamená, lex Babiš je pilotní projekt. Já si myslím, že pan předseda hnutí ANO by si mohl blahopřát, protože v moderním zákonodárství byli pouze dva lidé, kvůli kterým byly přijímány urychleně zákony, a v obou dvou pří
***
Kdo: Poslanec Zbyněk Stanjura
Kde: 2013ps_stenprot_005schuz_s005004.htm
Kdy: 2014-01-21
Co: Děkuji za slovo. Vážený pane předsedo, kolegyně a kolegové, něco už řekli moji kolegové, něco doplním. Já myslím, že je to naprostá pravda, že jsme tady dneska jenom kvůli bodu 5, lex Babiš, ty ostatní jsou kouřová clona. Bohužel jsou tam podepsáni jiní opoziční politici a vytvářejí tu kouřovou clonu s těmi, kteří to předložili. Není to nic překvapivého a myslím, že takových chování jso
***
Kdo: P

In [46]:
df.filter(pl.col("datum").dt.year().is_between(1993,2013)).filter(pl.col("text").str.contains("(?i)[^\w]lex \w"))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_1168\3880432954.py:1: SyntaxWarning: invalid escape sequence '\w'
  df.filter(pl.col("datum").dt.year().is_between(1993,2013)).filter(pl.col("text").str.contains("(?i)[^\w]lex \w"))


mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool
"""Poslanec Vladimír Řezáč""","""1993ps_stenprot_011schuz_s011024.htm""",null,"""Vážený pane předsedo, vážený pane místopředsedo, kolegyně a kolegové, kolega Ivan Mašek řekl, že mu je ctí přednést jako zástupce předkladatele tento …",3,"""ps""",1993,11,24,1993-07-09,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""11. schůze""",true,true,true
"""Poslanec Jiří Payne""","""1993ps_stenprot_013schuz_s013019.htm""",null,"""Pane předsedající, pane ministře, vážená sněmovno, do jisté míry naváži na příspěvek pana kolegy Vrzala, protože vyvstává otázka financování našeho fi…",10,"""ps""",1993,13,19,1993-10-14,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""13. schůze""",true,true,true
"""Poslanec Miroslav Čapek""","""1993ps_stenprot_021schuz_s021002.htm""","""/sqw/hlasy.sqw?G=5310""","""Vážený pane předsedo, vážený pane ministře, dámy a pánové. Vláda České republiky předložila parlamentu k projednání návrh zákona o zpravodajských služ…",8,"""ps""",1993,21,2,1994-07-07,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""21. schůze""",true,true,true
"""Poslanec Vladimír Řezáč""","""1993ps_stenprot_025schuz_s025039.htm""","""/sqw/hlasy.sqw?G=4554""","""Vážený pane místopředsedo, vážené kolegyně a kolegové, projednáváme novelu základního civilního kodexu, lex generalis, který upravuje civilní sféru ce…",17,"""ps""",1993,25,39,1994-12-13,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""25. schůze""",true,true,true
"""Poslanec Viktor Dobal""","""1993ps_stenprot_034schuz_s034014.htm""","""/sqw/hlasy.sqw?G=2390""","""Vážený pane předsedající, vážený pane předsedo, vážené paní poslankyně, páni poslanci, dovolím si vznést několik pozměňovacích návrhů. Dříve, než to u…",3,"""ps""",1993,34,14,1995-09-27,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""34. schůze""",true,true,true
"""Poslanec Zdeněk Jičínský""","""1996ps_stenprot_003schuz_s003004.htm""","""/sqw/hlasy.sqw?G=5978""","""Pane předsedající, paní poslankyně, páni poslanci, býval jsem zvyklý mluvit v Parlamentě z patra a doufám, že se k tomuto dobrému zvyku budu moci vrát…",8,"""ps""",1996,3,4,1996-07-24,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""3. schůze""",true,true,true
"""Poslanec Zdeněk Jičínský""","""1996ps_stenprot_011schuz_s011031.htm""","""/sqw/hlasy.sqw?G=8471""","""Pane předsedo, vážené shromáždění, problematika referenda není na této půdě poprvé, nejednou se o ní jednalo již v průběhu minulého volebního období. …",15,"""ps""",1996,11,31,1997-05-16,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""11. schůze""",true,true,true
"""Poslanec Jiří Payne""","""1996ps_stenprot_025schuz_s025131.htm""","""/cgi-bin/win/sqw/hlasy.sqw?G=11995""","""Myslím si, že bychom mohli najít někdy v budoucnosti formulaci, která by to odlišila, nicméně teď jsme v takovém stavu, kdy máme jednu novelizaci, kte…",1,"""ps""",1996,25,131,1998-05-15,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""25. schůze""",true,true,false
"""Poslanec Václav Krása""","""1998ps_stenprot_009schuz_s009053.htm""","""/sqw/detail.sqw?o=3&id=79""","""Vážený pane předsedající, vážení členové vlády, dámy a pánové, dovolím si říci několik slov ke stanovisku vlády, které, jak víte, bylo k tomuto návrhu…",3,"""ps""",1998,9,53,1999-02-03,"""Parlament České republiky, Poslanecká sněmovna 1998 – 2002""","""9. schůze""",true,true,true


In [47]:
lexy_do_2013 = df.filter(pl.col('datum').dt.year().is_between(2010,2013)).filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)
lexy_do_2013

C:\Users\micha\AppData\Local\Temp\ipykernel_1168\3437701511.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  lexy_do_2013 = df.filter(pl.col('datum').dt.year().is_between(2010,2013)).filter(pl.col('text').str.contains(r' [Ll]ex ')).with_columns(pl.col("text").map_elements(najdi_lexy).alias("lex")).explode('lex').group_by("lex").len().sort(by="len",descending=True)


lex,len
str,u32
"""Lex Farský""",6
"""Lex specialis""",4
"""Lex Peksová""",1
"""Lex Šťovíček""",1
"""Lex Pekárek""",1
"""Lex Suchánek""",1
"""Lex Beneš""",1


In [48]:
hledej("lex Farský")

Kdo: Poslanec Josef Novotný st.
Kde: 2010ps_stenprot_019schuz_s019295.htm
Kdy: 2011-06-21
Co: u byly. Velice mě zklamala porada o hazardu, která proběhla minulý pátek po zasedání Sněmovny, kde mělo dojít k nějaké dohodě napříč. Zjistil jsem, že dohoda v podstatě směřuje k tomu, že je tady ten lex Farský nebo komplexní pozměňovák široce přijímaný, že vůbec Věci veřejné a jejich požadavky, aby bylo průhledněji nakládáno se zvýšenými odvody na veřejně prospěšné účely, nejsou respektovány, že nebude ods
***
Kdo: Poslanec Michal Doktor
Kde: 2010ps_stenprot_019schuz_s019298.htm
Kdy: 2011-06-21
Co: žit a prospěje spíše těm velkým nežli těm malým. Prosím vás o zvážení mých argumentů. Nepochybně je možné odpovědět na dnešní dilema i tím, že dříve nežli navrhované efekty toho, čemu se začalo říkat lex Farský, vstoupí v účinnost, resp. budou tlačit na reálný trh, bude tady daleko dříve návrh ministra financí a toto riziko bude překonáno. To je pro některé z vás jistým odlehčením. Já vás jen prosí